In [3]:
## Inserindo dados do atlas no data lake
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np



In [19]:
## Acessando pelo pacote da BD

df=bd.read_sql("SELECT ano,tema,  id_municipio,existencia_priorizacao_cargo_mulher FROM `basedosdados-projetos.republica.indicadores_munic_gerais` where ano=2018 and tema='recursos_humanos' ", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 11561.15rows/s]


In [6]:
# Equivalente ao table do R
pd.crosstab(index = df["existencia_priorizacao_cargo_mulher"],
                              columns="count")

col_0,count
existencia_priorizacao_cargo_mulher,
0,5413
1,152


In [7]:
df['existencia_priorizacao_cargo_mulher'].unique()

<IntegerArray>
[<NA>, 0, 1]
Length: 3, dtype: Int64

In [20]:
df = df.dropna()

In [10]:
len(df)

5565

In [21]:
df['existencia_priorizacao_cargo_mulher'] = np.where(df['existencia_priorizacao_cargo_mulher']== 0, "Não", "Sim")

In [22]:
## Trazendo nomes de municipios
municipio=bd.read_sql("select sigla_uf, id_municipio, nome from `basedosdados.br_bd_diretorios_brasil.municipio`", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)
df = df.merge(municipio, left_on='id_municipio', right_on='id_municipio')

Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 12927.64rows/s]


In [23]:
df

,ano,tema,id_municipio,existencia_priorizacao_cargo_mulher,sigla_uf,nome
0,2018,recursos_humanos,1100023,Não,RO,Ariquemes
1,2018,recursos_humanos,1100031,Não,RO,Cabixi
2,2018,recursos_humanos,1100049,Não,RO,Cacoal
3,2018,recursos_humanos,1100056,Não,RO,Cerejeiras
4,2018,recursos_humanos,1100064,Não,RO,Colorado do Oeste
...,...,...,...,...,...,...
5560,2018,recursos_humanos,5222005,Não,GO,Vianópolis
5561,2018,recursos_humanos,5222054,Não,GO,Vicentinópolis
5562,2018,recursos_humanos,5222203,Não,GO,Vila Boa
5563,2018,recursos_humanos,5222302,Não,GO,Vila Propício


In [24]:
 df.rename({'nome':'nome_municipio'}, inplace=True, axis=1)

In [25]:

df = df.reindex(columns=['ano', 'tema','sigla_uf', 'id_municipio','nome_municipio', 'existencia_priorizacao_cargo_mulher'])

In [26]:
 df.drop(['tema'], axis=1, inplace=True)

In [15]:
client = bigquery.Client()

In [16]:
dataset_ref = client.dataset('acoes_afirmativas')

In [32]:
df.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'nome_municipio',
       'existencia_priorizacao_cargo_mulher'],
      dtype='object')

In [18]:
df.dtypes

ano                                     Int64
id_municipio                           object
nome_municipio                         object
existencia_priorizacao_cargo_mulher    object
dtype: object

In [37]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação daquele município.'),
 bigquery.SchemaField('id_municipio','STRING',description='ID do IBGE para aquele município.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
 bigquery.SchemaField('existencia_priorizacao_cargo_mulher','STRING',description='Se existe ou não existe política de priorização para mulheres em cargos de chefia e assessoramento.')
 ]

In [34]:
table_ref = dataset_ref.table('MUNIC_politica_mulheres')

In [38]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [39]:
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=f58b0c57-467b-4b5f-b4ef-cde6002690de>